# Вебинар 4. Домашнее задание

In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, item_features, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])[
            'quantity'].count().reset_index()
        self.top_purchases.sort_values(
            'quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]
        
        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby(
            'item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values(
            'quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[
            self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        # Топ покупок по товарам СТМ
        self.top_ctm_purchases = item_features[item_features['brand'] == 'Private'].item_id.unique()
        self.top_ctm_purchases = np.array(self.overall_top_purchases)[
            np.isin(self.overall_top_purchases, self.top_ctm_purchases)]
        self.top_ctm_purchases = self.top_ctm_purchases.tolist()
        
        
        # User-item матрица
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(
            self.user_item_matrix)

        # Словарь {item_id: 0/1}. 0/1 - факт принадлежности товара к СТМ
        self.item_list = data['item_id'].unique().tolist()
        self.item_id_to_ctm = {
            i: 1 if i in self.top_ctm_purchases else 0 for i in self.item_list}

        # Own recommender обучается до взвешивания матрицы
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data):

        # your_code
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)
        
        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(
            factors=n_factors,
            regularization=regularization,
            iterations=iterations,
            num_threads=num_threads
        )
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def get_similar_item(self, item_id, filter_ctm):
        """Находит товар, похожий на item_id"""

        # Рекомендуем 10 товаров
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=10)
        # Берем второй (не товар из аргумента метода)
        top_rec = recs[1][0]

        # Если есть фильтр по СТМ товарам, то из рекомендованных товаров выбираем первый СТМ товар
        if filter_ctm:
            recs = [r[0] for r in recs]  # оставляем только id товаров
            recs = recs[1:]  # убираем текущий товар
            # убираем не СТМ товары
            recs = np.aray(recs)[np.isin(recs, self.top_ctm_purchases)]
            top_rec = recs[0] if len(recs) > 0 else self.top_ctm_purchases[0]

        return self.id_to_itemid[top_rec]

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        recs = self.own_recommender.recommend(
            userid=self.userid_to_id[1],
            user_items=csr_matrix(self.user_item_matrix).tocsr(),
            N=N,
            filter_already_liked_items=False,
            filter_items=[self.itemid_to_id[999999]],
            recalculate_user=True
        )

        res = [self.id_to_itemid[rec[0]] for rec in recs]

        return res

    def get_similar_items_recommendation(self, user, filter_ctm=True, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        # Практически полностью реализовали на прошлом вебинаре
        # Не забывайте, что нужно учесть параметр filter_ctm

        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(
            N)
        res = top_users_purchases['item_id'] \
            .apply(lambda x: self.get_similar_item(x, filter_ctm=filter_ctm)) \
            .tolist()

        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        # your_code
        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(
            self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

        return res

In [2]:
# проврка подгрузки
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender